In [1]:
# Standard libraries
import xarray as xr
import numpy as np
import pandas as pd
# import os
# from glob import glob
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.pyplot as plt
%matplotlib inline
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
import seaborn as sns
import iris
from iris.pandas import as_cubes
import sys
import math
import os
import glob

from datetime import datetime
from cartopy.util import add_cyclic_point
import gc
import imageio.v2
from IPython import display
# import netCDF4
from global_land_mask import globe
# # Import tobac itself:
import tobac
from merge_split import merge_split_MEST, merge_split_detector
# Disable a few warnings:
import warnings
warnings.filterwarnings('ignore', category=UserWarning, append=True)
warnings.filterwarnings('ignore', category=RuntimeWarning, append=True)
warnings.filterwarnings('ignore', category=FutureWarning, append=True)
warnings.filterwarnings('ignore',category=pd.io.pytables.PerformanceWarning)

/glade/work/noteng/conda-envs/tobac-env/lib/python3.11/site-packages/tobac/utils/internal.py:448: UserWarning: Numba not able to be imported; periodic boundary calculations will be slower.Exception raised: ModuleNotFoundError("No module named 'numba'")
  warnings.warn(
/glade/work/noteng/conda-envs/tobac-env/lib/python3.11/site-packages/tobac/utils/internal.py:448: UserWarning: Numba not able to be imported; periodic boundary calculations will be slower.Exception raised: ModuleNotFoundError("No module named 'numba'")
  warnings.warn(


In [2]:
%%time
path = '/glade/u/home/noteng/work/masters-research/data/'
file = 'march13-march14.nc'
data = xr.open_dataset(path+file)
data = data.sel(time=slice('2020-03-13T04:00:00.000000000', '2020-03-14T05:00:00.000000000'))
data.close()

CPU times: user 54.5 ms, sys: 23.3 ms, total: 77.7 ms
Wall time: 430 ms


In [3]:
# equivalent_reflectivity_factor = data['equivalent_reflectivity_factor'][:,450:580,256:771] #Based on longitude and latitude of Andoya and Norwegian Sea
equivalent_reflectivity_factor = data['equivalent_reflectivity_factor'][:,250:650,450:850] #Based on longitude and latitude of Andoya and Norwegian Sea
# equivalent_reflectivity_factor = data['equivalent_reflectivity_factor'][:,330:580,660:780] #### hdm1 and hdm2
# equivalent_reflectivity_factor = data['equivalent_reflectivity_factor']
equivalent_reflectivity_factor

<xarray.DataArray 'equivalent_reflectivity_factor' (time: 301, Yc: 400, Xc: 400)>
[48160000 values with dtype=float32]
Coordinates:
  * time     (time) datetime64[ns] 2020-03-13T04:00:00 ... 2020-03-14T05:00:00
  * Xc       (Xc) float32 -8.3e+04 -8.2e+04 -8.1e+04 ... 3.15e+05 3.16e+05
  * Yc       (Yc) float32 -2.06e+06 -2.061e+06 ... -2.458e+06 -2.459e+06
    lon      (Yc, Xc) float32 ...
    lat      (Yc, Xc) float32 ...
Attributes:
    standard_name:  equivalent_reflectivity_factor
    long_name:      Radar Reflectivity
    units:          dBZ
    grid_mapping:   projection_lambert

In [4]:
# load Mask_ERF_files
Mask_ERF = iris.load('../saved-files/threshold-all/Mask_ERF_iris-all-all.nc')[0]
Mask_ERF

<iris 'Cube' of segmentation_mask / (1) (time: 301; -- : 400; -- : 400)>

In [5]:
tracks = pd.read_csv('../saved-files/threshold-all/cells-allkm-KAZR-thres-all-all.csv')
tracks

,frame,idx,hdim_1,hdim_2,num,threshold_value,feature,time,timestr,projection_y_coordinate,projection_x_coordinate,latitude,longitude,cell,time_cell,distance,mean dBZ,max dBZ,area($km^2$),cloud_id
0,0,453,121.000000,324.705104,2,0,5,2020-03-13 04:00:00,2020-03-13 04:00:00,-2.181000e+06,241705.104259,70.248601,16.323888,5,0 days 00:00:00,125.680339,13.245867,34.567482,132.310998,1
1,0,456,135.877490,273.231217,7,0,6,2020-03-13 04:00:00,2020-03-13 04:00:00,-2.195877e+06,190231.216954,70.159887,14.951233,6,0 days 00:00:00,116.846876,13.245867,34.567482,331.269006,2
2,0,471,168.000000,286.000000,1,0,10,2020-03-13 04:00:00,2020-03-13 04:00:00,-2.228000e+06,203000.000000,69.858376,15.206022,10,0 days 00:00:00,81.936672,13.245867,34.567482,65.658469,3
3,0,481,203.473824,247.478730,4,0,13,2020-03-13 04:00:00,2020-03-13 04:00:00,-2.263474e+06,164478.729588,69.565035,14.156179,13,0 days 00:00:00,76.275220,13.245867,34.567482,327.324894,4
4,0,482,204.000000,286.176995,2,0,14,2020-03-13 04:00:00,2020-03-13 04:00:00,-2.264000e+06,203176.994856,69.531731,15.128128,14,0 days 00:00:00,48.610059,13.245867,34.567482,171.124975,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55639,300,555,178.000000,358.012383,2,20,66372,2020-03-14 05:00:00,2020-03-14 05:00:00,-2.238000e+06,275012.383147,69.698076,17.005561,15276,0 days 00:10:00,80.687235,14.702386,40.106802,208.928144,7117
55640,300,575,285.000000,350.000000,1,20,66383,2020-03-14 05:00:00,2020-03-14 05:00:00,-2.345000e+06,267000.000000,68.738228,16.495682,15277,0 days 00:10:00,55.407771,14.702386,40.106802,214.931734,7118
55641,300,576,286.727785,374.604901,6,20,66384,2020-03-14 05:00:00,2020-03-14 05:00:00,-2.346728e+06,291604.900654,68.696045,17.083278,15278,0 days 00:10:00,74.819683,14.702386,40.106802,494.549355,7119
55642,300,539,295.648140,392.416781,10,15,66361,2020-03-14 05:00:00,2020-03-14 05:00:00,-2.355648e+06,309416.781206,68.594721,17.483020,15279,0 days 00:10:00,94.427981,14.702386,40.106802,115.430654,7120


In [6]:
# sorting out values based on cell_id(provided by TOBAC by default) and cloud_id(manually created)
tracks = tracks.sort_values(by=['cell', 'cloud_id']).reset_index(drop=True)
tracks

,frame,idx,hdim_1,hdim_2,num,threshold_value,feature,time,timestr,projection_y_coordinate,projection_x_coordinate,latitude,longitude,cell,time_cell,distance,mean dBZ,max dBZ,area($km^2$),cloud_id
0,0,453,121.000000,324.705104,2,0,5,2020-03-13 04:00:00,2020-03-13 04:00:00,-2.181000e+06,241705.104259,70.248601,16.323888,5,0 days 00:00:00,125.680339,13.245867,34.567482,132.310998,1
1,1,581,124.000000,327.838844,2,5,374,2020-03-13 04:05:00,2020-03-13 04:05:00,-2.184000e+06,244838.843751,70.218311,16.396476,5,0 days 00:05:00,122.977127,12.974534,35.577388,158.174980,1
2,2,716,127.000000,331.000000,1,10,780,2020-03-13 04:10:00,2020-03-13 04:10:00,-2.187000e+06,248000.000000,70.187965,16.469553,5,0 days 00:10:00,120.373766,12.780750,34.063904,272.576605,1
3,3,578,128.746166,333.673351,6,5,1063,2020-03-13 04:15:00,2020-03-13 04:15:00,-2.188746e+06,250673.350542,70.169425,16.533521,5,0 days 00:15:00,119.040644,12.708913,36.584485,247.709497,1
4,4,611,133.000000,334.000000,1,5,1406,2020-03-13 04:20:00,2020-03-13 04:20:00,-2.193000e+06,251000.000000,70.130630,16.529381,5,0 days 00:20:00,114.859159,12.845599,38.092506,94.508120,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55639,299,476,293.326516,390.199380,15,15,66254,2020-03-14 04:55:00,2020-03-14 04:55:00,-2.353327e+06,307199.379687,68.618348,17.437243,15279,0 days 00:05:00,91.303475,15.240493,41.116443,117.421341,7120
55640,300,539,295.648140,392.416781,10,15,66361,2020-03-14 05:00:00,2020-03-14 05:00:00,-2.355648e+06,309416.781206,68.594721,17.483020,15279,0 days 00:10:00,94.427981,14.702386,40.106802,115.430654,7120
55641,298,555,260.000000,211.000000,1,25,66190,2020-03-14 04:50:00,2020-03-14 04:50:00,-2.320000e+06,128000.000000,69.072189,13.157945,15281,0 days 00:00:00,100.574806,15.301029,39.603226,100.492397,7121
55642,299,531,260.153314,213.000000,3,25,66288,2020-03-14 04:55:00,2020-03-14 04:55:00,-2.320153e+06,130000.000000,69.069779,13.206974,15281,0 days 00:05:00,98.661443,15.240493,41.116443,106.461253,7121


### _Finding cells that merge or split_

In [7]:
# %%time
# merge_split_detector(TRACK=tracks, dxy=1000, cell=4510)

## _Concentrate now on cell-evolution_ 

In [8]:
# Getting the unique cells
unique_cell_ids = np.unique(tracks['cell'].values)
unique_cell_ids

array([    5,     6,    10, ..., 15278, 15279, 15281])

In [9]:
%%time
# all masked cell in iris cube
masked_tracked_cell = []
for i, unique_value in enumerate(unique_cell_ids):
    # get masked cell based on cell id
    get_mask_cell = tobac.utils.mask_cell(mask=Mask_ERF, cell=unique_value, track=tracks, masked=False) 
    masked_tracked_cell.append(get_mask_cell)

CPU times: user 18.4 s, sys: 276 ms, total: 18.7 s
Wall time: 19.8 s


In [10]:
%%time
# Convert the segmented/mask cells from iris cube to xrray
masked_values_to_xarray = []
for i, items in enumerate(masked_tracked_cell):
    mask_convert_xr = xr.DataArray.from_iris(masked_tracked_cell[i]) # convert iris cube to xarray data array
    masked_values_to_xarray.append(mask_convert_xr) # append the results to list
    # print(mask_convert_xr)

CPU times: user 1min 5s, sys: 21.5 s, total: 1min 26s
Wall time: 1min 32s


In [11]:
# getting the cell information
cell1 = tracks[tracks['cell'] == 5754]
cell1

,frame,idx,hdim_1,hdim_2,num,threshold_value,feature,time,timestr,projection_y_coordinate,projection_x_coordinate,latitude,longitude,cell,time_cell,distance,mean dBZ,max dBZ,area($km^2$),cloud_id
21509,76,644,190.000000,278.000000,1,0,24217,2020-03-13 10:20:00,2020-03-13 10:20:00,-2.250000e+06,195000.000000,69.665291,14.953258,5754,0 days 00:00:00,64.941205,16.047572,40.610380,0.995001,2743
21510,77,1165,198.000000,280.000000,1,20,24653,2020-03-13 10:25:00,2020-03-13 10:25:00,-2.258000e+06,197000.000000,69.591125,14.986165,5754,0 days 00:05:00,57.039202,15.974404,39.102244,125.358252,2743
21511,78,1239,200.747427,282.000000,2,20,24931,2020-03-13 10:30:00,2020-03-13 10:30:00,-2.260747e+06,199000.000000,69.564608,15.030438,5754,0 days 00:10:00,53.623822,15.958169,40.106802,107.451090,2743
21512,79,1185,203.946177,284.946177,2,20,25207,2020-03-13 10:35:00,2020-03-13 10:35:00,-2.263946e+06,201946.176554,69.533216,15.097348,5754,0 days 00:15:00,49.310795,15.878134,39.605794,55.717393,2743
21513,80,1245,202.000000,288.000000,1,25,25524,2020-03-13 10:40:00,2020-03-13 10:40:00,-2.262000e+06,205000.000000,69.548386,15.178442,5754,0 days 00:20:00,49.443731,15.876115,40.610380,111.430327,2743
21514,81,1218,205.000000,290.000000,1,20,25747,2020-03-13 10:45:00,2020-03-13 10:45:00,-2.265000e+06,207000.000000,69.519516,15.221797,5754,0 days 00:25:00,45.822608,16.048289,39.102244,79.595499,2743
21515,82,1196,207.000000,293.000000,1,20,26000,2020-03-13 10:50:00,2020-03-13 10:50:00,-2.267000e+06,210000.000000,69.498871,15.292401,5754,0 days 00:30:00,42.647130,16.519295,41.617532,99.495657,2743
21516,83,1305,209.000000,294.393280,3,25,26324,2020-03-13 10:55:00,2020-03-13 10:55:00,-2.269000e+06,211393.279811,69.479556,15.322645,5754,0 days 00:35:00,40.216473,16.479151,40.109338,99.494580,2743
21517,84,1224,210.512071,300.257348,7,20,26524,2020-03-13 11:00:00,2020-03-13 11:00:00,-2.270512e+06,217257.347964,69.460822,15.465792,5754,0 days 00:40:00,36.558294,16.305907,41.113953,0.994743,2743
21518,85,1289,212.000000,304.000000,1,25,26841,2020-03-13 11:05:00,2020-03-13 11:05:00,-2.272000e+06,221000.000000,69.444054,15.555747,5754,0 days 00:45:00,34.045311,16.169336,41.116443,147.248815,2743


In [12]:
# getting the cell-id(just one)
cell_number = cell1['cell'].values[0]
cell_number

5754

In [13]:
%%time
# cells that merged or splitted
# merge_split_detector(TRACK=tracks, dxy=1000, cell=cell_number)

CPU times: user 4 µs, sys: 1 µs, total: 5 µs
Wall time: 8.11 µs


In [14]:
# mask individual cell
mask_cell1 = tobac.utils.mask_cell(Mask_ERF, cell_number, tracks)
mask_cell1

<iris 'Cube' of segmentation_mask / (1) (time: 301; -- : 400; -- : 400)>

In [15]:
# converting cell to xarray dataarray from iris cube
convert_cell1 = xr.DataArray.from_iris(mask_cell1)
convert_cell1

<xarray.DataArray 'segmentation_mask' (time: 301, dim_1: 400, dim_2: 400)>
dask.array<filled, shape=(301, 400, 400), dtype=int32, chunksize=(150, 400, 400), chunktype=numpy.ndarray>
Coordinates:
  * time     (time) datetime64[ns] 2020-03-13T04:00:00 ... 2020-03-14T05:00:00
    lat      (dim_1, dim_2) float32 ...
    lon      (dim_1, dim_2) float32 ...
Dimensions without coordinates: dim_1, dim_2
Attributes:
    long_name:    segmentation_mask
    Conventions:  CF-1.7

In [16]:
%%time
for idx, i in enumerate(cell1['frame'].values[:]):
    ## center points and extent of map ###
    latA = 69.141281 #latitude of COMBLE site
    lonA = 15.684166-1 #longitude of COMBLE site -1
    xm, ym = 6,2.75 # extent from center point in lon and lat
    fig, ax = plt.subplots(1,1,figsize=(10, 9), subplot_kw={'projection': ccrs.Orthographic(lonA,latA)})
    extent = (lonA-xm+3.3, lonA+xm-3.7, latA-ym+2, latA+ym-1.5) # define extent map
    ax.set_extent(extent) # set extent of map
    ax.coastlines(resolution='10m', color='black') # plot coastlines with high resolution: 10m
    ax.set_facecolor('white')
    ax.add_feature(cfeature.LAND, linewidth=1.5, alpha=0.5)

    # add grid lines
    gl = ax.gridlines(x_inline=False, alpha=0.3, color='white'); gl.bottom_labels=True; gl.left_labels=True
    gl.xformatter = LONGITUDE_FORMATTER; gl.yformatter = LATITUDE_FORMATTER
    gl.xlabel_style = {'rotation': 45}; gl.ylabel_style = {'color': 'black'}

    #change time to moth, day, year, hours and minutes
    dt = masked_values_to_xarray[idx]['time'].dt.strftime('%m-%d-%Y %H:%M').values  ### array
    dt_saved_files_format = masked_values_to_xarray[idx]['time'].dt.strftime('%m-%d-%Y %H_%M').values  ### array
    

#     #     # fill in the place where there is cloud with 1 and place with no cloud to be zero
    cloud = np.asarray(convert_cell1[i], dtype='float')
    # cloud = np.asarray(convert_660[i], dtype='float')
    # print(type(cloud))
    invalid_cloud = np.greater(cloud, 0)
    cloud[invalid_cloud] = 1
    invalid_cloud1 = np.less(cloud,1)
    cloud[invalid_cloud1] = np.nan
    

    cs=ax.pcolormesh(convert_cell1['lon'],
                convert_cell1['lat'], 
                # cloud*equivalent_reflectivity_factor.values[i],
                cloud*equivalent_reflectivity_factor.values[i],
                cmap='nipy_spectral',
                transform=ccrs.PlateCarree(),
                vmin=-30,
                vmax=30
                )
#     # display each id on cell
    frame=cell1[cell1['frame']==i]
    longi = frame['longitude'].values[0]
    lati = frame['latitude'].values[0]
    cloud_id = frame['cloud_id'].values[0]
    cell_id = frame['cell'].values[0]
    # print(f"cell-id of {cell_id}, now id {cloud_id}")
    
    
    
    # check if area has value of nan
    # if not math.isnan(frame['area($km^2$)'].values[0]):
#     fontdict={'fontweight': 'extra bold'}
#     ax.text(longi, lati+0.1, cell_id, transform=ccrs.PlateCarree(), fontsize=13, color='white', fontdict=fontdict)
    
    fontdict={'fontweight': 'extra bold'}
    ax.text(longi, lati, cell_id, transform=ccrs.PlateCarree(), fontsize=9.7, color='black', fontdict=fontdict)



    ax.plot(lonA+1,latA, color='red', marker='*', markersize = 7.5,transform=ccrs.PlateCarree(), zorder=7) # plot red star at location of Andenes
    ax.set_title(f"Nordic Radar Mosaic: {dt[i]} UTC", fontweight='bold', fontsize=16, pad=11)  # title of figure


    # Add a colorbar axis at the right of the graph
    # cbar_ax = fig.add_axes([0.85, 0.13, 0.047, 0.75]) #lbwh
    cbar_ax = fig.add_axes([0.85, 0.11, 0.047, 0.77]) #lbwh


    cbar=fig.colorbar(cs, cax=cbar_ax, orientation='vertical')
    cbar.set_label("Reflectivity Factor [dBZ]", fontsize = 15)
    
    
    fig.savefig(f'./evolution-cells-final/cell11//{dt_saved_files_format[i]}.png', dpi=300)
    plt.close()



#     dirr = sorted(os.listdir('./new//'))
#     if f'{dt[i]}.png' in dirr:
#         # print(f'{dt_saved_files_format[i]}.png already exist')
#         pass
#         plt.close()
#     else:
#         fig.savefig(f'./new//{dt_saved_files_format[i]}.png', dpi=100)
#         plt.close()
# print('\U0001f600\U0001f600\U0001f600\U0001f600EXECUTION DONE!!!\U0001f600\U0001f600\U0001f600\U0001f600')
# # gc.collect()

CPU times: user 41 s, sys: 2.35 s, total: 43.3 s
Wall time: 48.5 s


In [17]:
%%time
# import glob
# Build GIF
imgs = sorted(glob.glob("./evolution-cells-final/cell11*.png"))
with imageio.get_writer('./evolution-cells-final/cell11/cell11.gif', mode='I', duration=0.7) as writer:
    for filename in imgs:
        image = imageio.v2.imread(filename)
        writer.append_data(image)

CPU times: user 7.03 ms, sys: 0 ns, total: 7.03 ms
Wall time: 7.39 ms
